In [1]:
from automl import AutoML

In [2]:
import openml

task = openml.tasks.get_task(31)

# estimation_parameters — словарь, но значения там строки
params = task.estimation_parameters
n_folds = int(params.get("number_folds", 1))
n_repeats = int(params.get("number_repeats", 1))

print("folds:", n_folds, "repeats:", n_repeats)

splits = []

for repeat in range(n_repeats):
    for fold in range(n_folds):
        train_idx, test_idx = task.get_train_test_split_indices(
            repeat=repeat,
            fold=fold
        )
        splits.append({
            "repeat": repeat,
            "fold": fold,
            "train_idx": train_idx,
            "test_idx": test_idx,
        })

# пример использования
dataset = task.get_dataset()
X, y, categorical, feature_names = dataset.get_data(target=dataset.default_target_attribute)

folds: 10 repeats: 1


In [3]:
from sklearn.metrics import roc_auc_score


automl = AutoML(
    task='classification',
    use_preprocessing_pipeline=False,
    feature_selector_type=None,
    use_val_test_pipeline=False,
    auto_models_init_kwargs = {
        "metric": "roc_auc",
        "time_series": False,
        "models_list": ["all"],
        "blend": True,
        "stack": True
    },
    n_jobs=6, 
    random_state=0,
    )
automl = automl.fit(
    X, y, 
    auto_model_fit_kwargs = {
        "tuning_timeout": 10
    }
)

[2025-11-14 18:00:44,527] - [   MODEL    ] - 1 out of 10. type
[2025-11-14 18:00:44,559] - [   START    ] - Working with type
[2025-11-14 18:00:44,562] - [   START    ] - Tuning type


[2025-11-14 18:00:49,684] - [   OPTUNA   ] - Trial 0. New best score 0.6019285714285714 with parameters {'max_iter': 8277, 'fit_intercept': True, 'C': 1.8590843630169627, 'solver': 'saga', 'penalty': 'elasticnet', 'l1_ratio': 0.9636627605010293}
[2025-11-14 18:00:53,279] - [   OPTUNA   ] - Trial 1. New best score 0.6019523809523809 with parameters {'max_iter': 5813, 'fit_intercept': True, 'C': 2.5601615250028704, 'solver': 'saga', 'penalty': None}
[2025-11-14 18:00:57,145] - [   OPTUNA   ] - Trial 2. New best score 0.6020476190476189 with parameters {'max_iter': 12506, 'fit_intercept': False, 'C': 744.2349368719797, 'solver': 'saga', 'penalty': None}
[2025-11-14 18:00:57,147] - [   OPTUNA   ] - 3 trials completed
[2025-11-14 18:00:57,175] - [BEST PARAMS ] - {'random_state': 0, 'n_jobs': 6, 'verbose': 0, 'label_encoders': {'checking_status': OrdinalEncoder(encoded_missing_value=-1, handle_unknown='use_encoded_value',
               unknown_value=-1), 'credit_history': OrdinalEncoder(enc

TypeError: LogisticRegression.__init__() got an unexpected keyword argument 'label_encoders'

In [ ]:

test_predictions = automl.predict(test.drop(columns=[target_name,]))
test_score = roc_auc_score(test[target_name], test_predictions[:, 1])